In [6]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
import json


movies = pd.read_csv('test4.csv')
print(movies.shape)
movies.head(2)

(959, 15)


,pk,adult,backdrop_path,original_language,overview,popularity,poster_path,released_date,title,vote_count,vote_average,genres,is_active,director,actors
0,526896,False,/gG9fTyDL03fiKnOpf2tr01sncnt.jpg,en,희귀혈액병을 앓고 있는 생화학자 모비우스는 동료인 마르틴과 함께 치료제 개발에 몰두...,12854.042,/8uiV88A5eB7PjMTtY6lQBUZ0Cpl.jpg,2022-03-30,모비우스,1091.0,6.4,"['Action', 'Science Fiction', 'Fantasy']",True,Pietro Scalia,"['Jared Leto', 'Matt Smith', 'Adria Arjona', '..."
1,752623,False,/A3bsT0m1um6tvcmlIGxBwx9eAxn.jpg,en,전설의 트레저를 차지하기 위해 재벌 페어팩스(다니엘 래드클리프)는 유일한 단서를 ...,11818.362,/8kZapNBZYGJu7AUbJVBDMmQgO1D.jpg,2022-03-24,로스트 시티,738.0,6.8,"['Action', 'Adventure', 'Comedy', 'Romance']",True,Marlene Stewart,"['Sandra Bullock', 'Channing Tatum', 'Daniel R..."


In [7]:
movies_df = movies[['pk', 'popularity', 'title','vote_count', 'vote_average', 'genres', 'director']]
movies_df.head(2)

,pk,popularity,title,vote_count,vote_average,genres,director
0,526896,12854.042,모비우스,1091.0,6.4,"['Action', 'Science Fiction', 'Fantasy']",Pietro Scalia
1,752623,11818.362,로스트 시티,738.0,6.8,"['Action', 'Adventure', 'Comedy', 'Romance']",Marlene Stewart


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from ast import literal_eval

# 문자열로 된 데이터를 literal_evalw로 변경
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
# genres의 각 단어들을 하나의 문장으로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))


count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])


  (0, 0)	1
  (0, 188)	1
  (0, 117)	1
  (0, 104)	1
  (0, 12)	1
  (0, 189)	1
  (0, 124)	1
  (1, 0)	1
  (1, 14)	1
  (1, 40)	1
  (1, 175)	1
  (1, 1)	1
  (1, 17)	1
  (1, 53)	1
  (2, 0)	1
  (2, 188)	1
  (2, 117)	1
  (2, 12)	1
  (2, 189)	1
  (2, 14)	1
  (2, 40)	1
  (2, 90)	1
  (2, 121)	1
  (2, 47)	1
  (2, 92)	1
  :	:
  (952, 72)	1
  (952, 62)	1
  (952, 86)	1
  (953, 90)	1
  (953, 27)	1
  (953, 32)	1
  (954, 190)	1
  (954, 138)	1
  (954, 72)	1
  (954, 146)	1
  (954, 81)	1
  (955, 175)	1
  (955, 72)	1
  (955, 84)	1
  (956, 72)	1
  (956, 129)	1
  (956, 80)	1
  (957, 14)	1
  (957, 72)	1
  (957, 19)	1
  (958, 104)	1
  (958, 14)	1
  (958, 90)	1
  (958, 21)	1
  (958, 110)	1
(959, 214)


In [10]:
# 코사인 유사돟를 이용해서 유사도를 찾기
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(959, 959)
[[1.         0.14285714 0.56980288 ... 0.         0.         0.16903085]
 [0.14285714 1.         0.34188173 ... 0.         0.21821789 0.16903085]]


In [16]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

In [18]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['pk']==title_name]

    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]
    
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 526896, 10)
similar_movies[['title', 'vote_average']]

[[  0 522 369 125 176 207 242 817 813 265]]


,title,vote_average
0,모비우스,6.4
522,매트릭스,8.2
369,인저스티스,7.7
125,고질라 VS. 콩,7.8
176,내일의 전쟁,8.0
207,쥬라기 헌트,5.0
242,아이언맨 2,6.8
817,가디언즈 오브 갤럭시,7.9
813,스타워즈 에피소드 6: 제다이의 귀환,7.9
265,드래곤볼Z: 신들의 전쟁,6.8


In [19]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
379,쇼생크 탈출,8.7,NaN
380,용감한 자가 신부를 데려가리,8.7,NaN
381,대부,8.7,NaN
384,Cosas imposibles,8.6,NaN
156,에브리씽 에브리웨어 올 앳 원스,8.6,NaN
382,쉰들러 리스트,8.6,NaN
383,대부 2,8.6,NaN
385,에브리씽 에브리웨어 올 앳 원스,8.6,NaN
386,동급생,8.6,NaN
402,시네마 천국,8.5,NaN


In [20]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']

    return ((v/(v+m)) * R) + ((m / (v + m)) * C)

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
movies_df.head(2)

,pk,popularity,title,vote_count,vote_average,genres,director,genres_literal,weighted_vote
0,526896,12854.042,모비우스,1091.0,6.4,"[Action, Science Fiction, Fantasy]",Pietro Scalia,Action Science Fiction Fantasy,7.090612
1,752623,11818.362,로스트 시티,738.0,6.8,"[Action, Adventure, Comedy, Romance]",Marlene Stewart,Action Adventure Comedy Romance,7.321145


In [21]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['pk'] == title_name]
    title_index = title_movie.index.values

    similar_indexes = sorted_ind[title_index, : (top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)

    similar_indexes = similar_indexes[similar_indexes != title_index]

    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 526896, 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
522,매트릭스,8.2,NaN
369,인저스티스,7.7,NaN
125,고질라 VS. 콩,7.8,NaN
176,내일의 전쟁,8.0,NaN
207,쥬라기 헌트,5.0,NaN
242,아이언맨 2,6.8,NaN
817,가디언즈 오브 갤럭시,7.9,NaN
813,스타워즈 에피소드 6: 제다이의 귀환,7.9,NaN
265,드래곤볼Z: 신들의 전쟁,6.8,NaN
282,캡틴 아메리카: 시빌 워,7.4,NaN
